In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/FusionModel

ModuleNotFoundError: No module named 'google'

In [ ]:
!pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0
!pip install transformers==4.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |████████████████████████████████| 22.1 MB 1.1 MB/s 
     |████████████████████████████████| 1.9 MB 62.4 MB/s 
     |████████████████████████████████| 7.6 MB 64.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
     

In [19]:
device = 'cpu'

In [3]:
from dataset import YouTubeDataset
dataset = YouTubeDataset()

In [2]:
from model import SimpleBert
model = SimpleBert()


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[ 0.1730,  0.2517,  0.0699, -0.0068]], grad_fn=<AddmmBackward0>).


In [4]:
import torch
checkpoint = torch.load("checkpoints/epoch200.pt",map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [ ]:
# model.base.requires_grad = False

In [14]:
loss_history = []

In [20]:
from sklearn.utils.class_weight import compute_class_weight
import torch
class_weights = compute_class_weight(class_weight='balanced', classes=torch.unique(dataset.label).numpy(), y=dataset.label.numpy())
print(class_weights)
class_weights = torch.tensor(class_weights, dtype=torch.float, device=device)
loss_fn = torch.nn.NLLLoss(weight = class_weights)
# from torch.nn.functional import cross_entropy
# loss_fn = cross_entropy

[0.45474138 0.85080645 2.39772727 4.79545455]


In [22]:
import torch
import math
import os
import time
import numpy as np

def train_model(model, train_data, loss_fn, learning_rate, lr_decay, batch_size, num_epochs, test_data=None, loss_history=None, device='cpu', isCheckpoint=False, isVerbose=True):
    model = model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), learning_rate
    )
    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda epoch: lr_decay ** epoch
    )

    # sample minibatch data
    iter_per_epoch = math.ceil(len(dataset) // batch_size)
    
    for i in range(num_epochs):
        start_t = time.time()
        local_hist = []
        correct_cnt = 0
        for j in range(iter_per_epoch):
            tokens, labels = dataset[j * batch_size: (j + 1) * batch_size]
            N = labels.shape[0]
            tokens = tokens.to(device)
            labels = labels.to(device)
            # groundTruth = torch.zeros((N,4),dtype=torch.long,device=device)
            # groundTruth[torch.arange(N),labels] = 1
            optimizer.zero_grad()

            digits = model(tokens)
            # print(groundTruth,digits)
            probs = torch.nn.LogSoftmax(dim=1)(digits)
            loss = loss_fn(probs,labels)
            loss.backward()

            local_hist.append(loss.item())
            optimizer.step()

        end_t = time.time()

        if loss_history:
            loss_mean = np.array(local_hist).mean()
            loss_history.append(loss_mean)
            
        if isVerbose:
            print(
                f"(Epoch {i} / {num_epochs}), time: {end_t - start_t:.1f}s"
            )
            print(f"/tTrain - loss: {loss_mean:.3f}, accuracy: {correct_cnt/len(train_data):.2f}, precision: {}")
        if i%100 == 0 and isCheckpoint:
          dir = "checkpoints"
          if not os.path.exists(dir):
            os.mkdir(dir)
          file = f"epoch{i}.pt"
          path = dir+'/'+file
          torch.save({
                      'epoch': i,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': loss_mean,
                      }, path)

        lr_scheduler.step()
        
train_model(model, dataset, loss_fn, learning_rate=1e-6, lr_decay=0.99, batch_size=10, num_epochs=500, loss_history = loss_history, isCheckpoint = True, isVerbose = True)

## 5-fold CV

In [10]:
from sklearn.model_selection import KFold
import torch

def train_model_cv5(model, dataset):
    epochs = 500
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(dataset):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, y_train = dataset[train_index]
        y_val, y_val = dataset[test_index]
        print(X_train, y_train)
        
train_model_cv5(model, dataset)

TRAIN: [ 43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114
 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132
 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 205 206 207 208 209 210] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
{'input_ids': tensor([[    0,  2387,   766,  ...,     1,     1,     1],
        [    0,  2847, 12402,  ...,     5,  2476,     2],
        [    0,  2847,    11,  ...,     6,

In [6]:
from train import eval

eval(model,dataset)

tensor([0])


KeyboardInterrupt: 